# Water Vapour

In [1]:
from datetime import datetime
from siphon.catalog import TDSCatalog
from datetime import datetime
import metpy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from metpy.plots import colortables
from metpy.plots import add_timestamp
import numpy             as np
import os                as os
import pandas             as pd


In [2]:
# %load solutions/data_url.py
total_frames = 45
os.system("rm -v ./sat_wv_temp_files/*")


# Cell content replaced by load magic replacement.

# Create variables for URL generation
image_date = datetime.utcnow().date()
region = 'CONUS'
channel = 8

# We want to match something like:
# https://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GOES16/Mesoscale-1/Channel08/20181113/catalog.html

# Construct the data_url string
data_url = ('https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/'
            f'CloudAndMoistureImagery/{region}/Channel{channel:02d}/current/catalog.xml')

# Print out your URL and verify it works!
print(data_url)


./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0006.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0011.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0016.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0021.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0026.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0031.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0036.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0041.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0046.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0051.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0056.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0101.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0106.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0111.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0116.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0121.png
./sat_wv_temp_files/Sat_WV_Loop_Image_2022-01-30_0126.png
./sat_wv_temp_

In [3]:
cat = TDSCatalog(data_url)

In [4]:
cat.datasets[1:total_frames]

[OR_ABI-L2-CMIPC-M6C08_G16_s20220300231170_e20220300231170_c20220300231170.nc,
 OR_ABI-L2-CMIPC-M6C08_G16_s20220300226170_e20220300226170_c20220300226170.nc,
 OR_ABI-L2-CMIPC-M6C08_G16_s20220300221170_e20220300221170_c20220300221170.nc,
 OR_ABI-L2-CMIPC-M6C08_G16_s20220300216170_e20220300216170_c20220300216170.nc,
 OR_ABI-L2-CMIPC-M6C08_G16_s20220300211170_e20220300211170_c20220300211170.nc,
 OR_ABI-L2-CMIPC-M6C08_G16_s20220300206170_e20220300206170_c20220300206170.nc,
 OR_ABI-L2-CMIPC-M6C08_G16_s20220300201170_e20220300201170_c20220300201170.nc,
 OR_ABI-L2-CMIPC-M6C08_G16_s20220300156170_e20220300156170_c20220300156170.nc,
 OR_ABI-L2-CMIPC-M6C08_G16_s20220300151170_e20220300151170_c20220300151170.nc,
 OR_ABI-L2-CMIPC-M6C08_G16_s20220300146170_e20220300146170_c20220300146170.nc,
 OR_ABI-L2-CMIPC-M6C08_G16_s20220300141170_e20220300141170_c20220300141170.nc,
 OR_ABI-L2-CMIPC-M6C08_G16_s20220300136170_e20220300136170_c20220300136170.nc,
 OR_ABI-L2-CMIPC-M6C08_G16_s20220300131170_e20220300

In [5]:
len(cat.datasets[1:total_frames])

29

In [6]:
for i in range(1,len(cat.datasets[1:total_frames])+1) : 

    dataset = cat.datasets[i]
    ds = dataset.remote_access(use_xarray=True)
    dat = ds.metpy.parse_cf('Sectorized_CMI')
    proj = dat.metpy.cartopy_crs
    x = dat['x']
    y = dat['y']


    tz='America/Denver'
    time_utc = datetime.strptime(ds.start_date_time, '%Y%j%H%M%S')
    valid_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d %H:%M:%S %Z")
    local_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).strftime("%Y-%m-%d %H:%M:%S %Z")

    file_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d_%H%M")


    print(valid_time,local_time)



    fig = plt.figure(figsize=(13, 8),
                           facecolor = 'white')

    plt.suptitle("GOES 16 Band 8 [6.2 µm Upper-level Water Vapor]",
                    fontsize=20)
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    ax.set_title(valid_time + "  (" + local_time+")",
                    fontsize=15)
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=2)
    ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':', edgecolor='black')
    ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=2, edgecolor='black')

    im = ax.imshow(dat, extent=(x.min(), x.max(), y.min(), y.max()), origin='upper')

    wv_norm, wv_cmap = colortables.get_with_range('WVCIMSS_r', 195, 265)
    im.set_cmap(wv_cmap)
    im.set_norm(wv_norm)
    plt.tight_layout()
    plt.savefig("./sat_wv_temp_files/Sat_WV_Loop_Image_"+file_time+".png")
    plt.close()
    



2022-01-30 02:31:17 UTC 2022-01-29 19:31:17 MST
2022-01-30 02:26:17 UTC 2022-01-29 19:26:17 MST
2022-01-30 02:21:17 UTC 2022-01-29 19:21:17 MST
2022-01-30 02:16:17 UTC 2022-01-29 19:16:17 MST
2022-01-30 02:11:17 UTC 2022-01-29 19:11:17 MST
2022-01-30 02:06:17 UTC 2022-01-29 19:06:17 MST
2022-01-30 02:01:17 UTC 2022-01-29 19:01:17 MST
2022-01-30 01:56:17 UTC 2022-01-29 18:56:17 MST
2022-01-30 01:51:17 UTC 2022-01-29 18:51:17 MST
2022-01-30 01:46:17 UTC 2022-01-29 18:46:17 MST
2022-01-30 01:41:17 UTC 2022-01-29 18:41:17 MST
2022-01-30 01:36:17 UTC 2022-01-29 18:36:17 MST
2022-01-30 01:31:17 UTC 2022-01-29 18:31:17 MST
2022-01-30 01:26:17 UTC 2022-01-29 18:26:17 MST
2022-01-30 01:21:17 UTC 2022-01-29 18:21:17 MST
2022-01-30 01:16:17 UTC 2022-01-29 18:16:17 MST
2022-01-30 01:11:17 UTC 2022-01-29 18:11:17 MST
2022-01-30 01:06:17 UTC 2022-01-29 18:06:17 MST
2022-01-30 01:01:17 UTC 2022-01-29 18:01:17 MST
2022-01-30 00:56:17 UTC 2022-01-29 17:56:17 MST
2022-01-30 00:51:17 UTC 2022-01-29 17:51

In [7]:
##################################################
#
# Convert PNGs into an Animated GIF
#

os.system("convert -delay 15 " + 
          "./sat_wv_temp_files/Sat_WV_Loop_Image_*.png"  + 
          " " + 
          "./graphics_files/RealTime_SAT_WV_Loop.gif")


#
##################################################

0